In [76]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', None)

import pickle

import scipy.sparse as sp
from itertools import islice, cycle
from more_itertools import pairwise
from tqdm.auto import tqdm

#Load data

In [6]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/interactions_preprocessed.pickle')
df_users = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/users_preprocessed.pickle')
df_items = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/items_preprocessed.pickle')

df.shape, df_users.shape, df_items.shape

((1562505, 5), (137254, 3), (63758, 5))

In [7]:
df.head()

,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01
3,12811,301895,16,NaN,2018-01-01
4,5778,127872,100,NaN,2018-01-01


In [8]:
# сопоставим user_id индекс
users_inv_mapping = dict(enumerate(df['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

158325

In [9]:
# аналогично для item_id
items_inv_mapping = dict(enumerate(df['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

63758

In [10]:
# приведем все названия к нижнему регистру
df_items['title'] = df_items['title'].str.lower()
# заведем словарь с названиями книг
item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()
len(item_titles), item_titles[248031]

(63758, 'ворон-челобитчик')

In [11]:
# посмотрим на кол-во книг с фикс. названием
title_items = df_items.groupby('title')['id'].agg(list)
title_items

title
"бегущий по лезвию 2049", «между нами горы", «борг/макинрой", «жизнь впереди"            [277167]
"железная леди" маргарет тетчер - величайшая женщина хх века                             [203030]
"зверский детектив" и "боги манго", "мия", "заяц на взлетной полосе", "страница один"    [232845]
"рэд 2", «смурфики 2", «византия" и др.                                                  [123247]
"сабля, водка, конь гусарский". история гусаров                                          [173381]
                                                                                           ...   
…чума на оба ваши дома!                                                                  [341578]
№ 12, или история одного прекрасного юноши                                               [386131]
伦巴德人的故事                                                                                  [278588]
地球への旅                                                                                    [372584]
�baby blues�  

In [12]:
# распределение по длинам названий
title_count = title_items.map(len)
title_count.value_counts()

1     53608
2      3623
3       488
4       151
5        61
6        23
7        17
8         8
9         5
18        1
10        1
11        1
12        1
35        1
13        1
15        1
51        1
Name: id, dtype: int64

In [13]:
title_items[title_count > 1].tail()

title
яр                           [108212, 252893]
ярмарка тщеславия    [323949, 351537, 324231]
ярослав мудрый                [358342, 83942]
ярость тьмы                   [324469, 74554]
ящик пандоры                  [93562, 164956]
Name: id, dtype: object

In [14]:
df_items[df_items['title'] == '451 градус по фаренгейту']

,id,title,genres,authors,year
49284,99550,451 градус по фаренгейту,"Социальная фантастика,Зарубежная фантастика,Научная фантастика,Классика фантастики",Рэй Брэдбери,"1951, 1953, 1967"


In [15]:
df['rating'] = np.array(df['rating'].values, dtype=np.float32)

df.loc[df['item_id'].isin([44681, 162716])].groupby('item_id').agg({
    'progress': np.size,
    'rating': ['mean'],
    'start_date': ['min', 'max'],
})

progress    rating start_date           
            size      mean        min        max
item_id                                         
162716        11  4.666667 2018-01-16 2019-08-06

#Popularity recom

In [39]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N].astype(int)
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [40]:
all_days = (df['start_date'].unique().max() - df['start_date'].unique().min()).astype('timedelta64[D]').astype('int')
all_days

729

In [41]:
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/sample_submission.csv')
sub.columns = ['user_id', 'Predicted']
train_df = sub['user_id'].to_frame().merge(df, on='user_id', how='left')
sub.columns = ['Id', 'Predicted']

pop_model = PopularRecommender(days=729, dt_column='start_date')
pop_model.fit(train_df)

top10_recs = pop_model.recommend()
top10_recs

array([ 80576,  37493,  68956,  75731, 270299, 364794, 330824, 178529,
       104445, 211600])

In [42]:
list(map(item_titles.get, top10_recs))

['сила подсознания, или как изменить жизнь за 4 недели',
 'снеговик',
 'невеста смерти',
 'земное притяжение',
 'цена вопроса. том 1',
 'долина драконов. магическая практика',
 'струны волшебства. книга первая. страшные сказки закрытого королевства',
 'яблоки из сада шлицбутера',
 'чудовищное предложение',
 'магическая сделка']

In [43]:
pop_recs = pd.DataFrame({'user_id': train_df['user_id'].unique()})
top_N = 10
pop_recs['item_id'] = pop_model.recommend(pop_recs['user_id'], N=top_N)
pop_recs.head()

,user_id,item_id
0,10001,"[80576, 37493, 68956, 75731, 270299, 364794, 330824, 178529, 104445, 211600]"
1,10002,"[80576, 37493, 68956, 75731, 270299, 364794, 330824, 178529, 104445, 211600]"
2,100152,"[80576, 37493, 68956, 75731, 270299, 364794, 330824, 178529, 104445, 211600]"
3,100197,"[80576, 37493, 68956, 75731, 270299, 364794, 330824, 178529, 104445, 211600]"
4,100284,"[80576, 37493, 68956, 75731, 270299, 364794, 330824, 178529, 104445, 211600]"


In [92]:
def dump_recs_to_csv(recs_df, fp):
    pred = []
    ids = []
    for id, recomm in tqdm(recs_df.groupby('user_id')):
        ids += [id]
        pred += [' '.join(map(str, recomm['item_id'].values[0]))]

    my_pop_submission = pd.DataFrame({'Id': ids,
                                      'Predicted': pred})
    sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/sample_submission.csv')
    my_pop_submission = sub['Id'].to_frame().merge(my_pop_submission,
                                                        on='Id', how='left')
    my_pop_submission.to_csv(fp, index = False)

    return my_pop_submission

In [93]:
my_pop_submission = dump_recs_to_csv(pop_recs, 
                                     '/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/submissions/pop_submission')
my_pop_submission.head()

,Id,Predicted
0,10001,80576 37493 68956 75731 270299 364794 330824 178529 104445 211600
1,10002,80576 37493 68956 75731 270299 364794 330824 178529 104445 211600
2,100152,80576 37493 68956 75731 270299 364794 330824 178529 104445 211600
3,100197,80576 37493 68956 75731 270299 364794 330824 178529 104445 211600
4,100284,80576 37493 68956 75731 270299 364794 330824 178529 104445 211600


In [74]:
def get_map_at_N(recs_df, train_df, top_N=10, print_met=False):
    # переводим строку рекомменлдаций для каждого пользователя в вектор (Series)
    test_recs = recs_df.explode('item_id')
    # сопоставляем топ-рекоммендациям наш рейтинг 
    test_recs['rank'] = test_recs.groupby('user_id').cumcount() + 1
    # join(им) df, на котором обучался, с нашими рекоммендациями (тут сразу получаем какие книги рекоммендовали правильно,
    #                                                             какие не порекоммендовали -- Nan (т.к. left join))
    test_recs = train_df.set_index(['user_id', 'item_id']).join(test_recs.set_index(['user_id', 'item_id']))
    # отсортируем для каждого user(а) и item(а)
    test_recs = test_recs.sort_values(by=['user_id', 'rating'])

    # найдем сколько всего у данного user(а) книг
    test_recs['users_item_count'] = tqdm(test_recs.groupby(level='user_id', sort=False)['rank'].transform(np.size))
    # находим reciprocal_rank
    test_recs['reciprocal_rank'] = 1 / test_recs['rank']
    test_recs['reciprocal_rank'] = test_recs['reciprocal_rank'].fillna(0)
    # находим (Precision at rank) для каждого пользовтеля
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']

    # СЧИТАЕМ MEAN AVARAGE PRECISION at 10
    metrics = {f'map@{top_N}': [], f'MRR': []}
    # находим кол-во всех пользователей в выборке
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        if print_met:
            print(f'Precision@{k} = {(test_recs[hit_k] / k).sum() / users_count:.4f}')
            print(f"Recall@{k} = {(test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count:.4f}")
    # усредняем Avarage (Precision at rank)
    mapN = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    metrics[f'map@{top_N}'] += [mapN]
    # 
    mrr = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    metrics['MRR'] += [mrr]

    if print_met:
        print(f"MAP@{top_N} = {mapN}")
        print(f"MRR = {mrr}")

    return metrics

In [75]:
metrics = get_map_at_N(pop_recs, train_df,)
print(metrics)


{'map@10': [0.03727178267234922], 'MRR': [0.061716418295793704]}


In [94]:
pickle.dump(PopularRecommender, 
            open('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/solutions_metrics/PopularRecommender.pkl', 'bw'))